# ID를 기준으로 전처리를 진행한다.
* 전처리(그룹화) 과정을 넘기고 싶으면 "그룹데이터"를 검색

### errtype & errcode(상위 30개) 전처리
ID 별로 errtype과 errcode를 count해준다

In [1]:
import pandas as pd 
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 
import gc
import random
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder,StandardScaler, scale
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline


In [2]:
#train 데이터와 test데이터의 경향을 먼저 살펴보자
train_err=pd.read_csv('train_err_data.csv')
test_err=pd.read_csv('test_err_data.csv')

In [3]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [4]:
test_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [5]:
#train 과 test의 errtype& errcode(상위30개)를 살펴본 결과 겹친다

#train_err['errtype'].unique()
#train_err['errtype'].unique()
#train_err['errcode'].unique()
#train_err['errcode'].unique()


## train 전처리

### errtype 전처리 (errtype 별 ID기준 count)

In [7]:
# errtype 별 count를 하기위해 onehot인코딩을 해준다
train_err_incoded=pd.get_dummies(train_err,columns=['errtype'])
train_err_incoded

,user_id,time,model_nm,fwver,errcode,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,20201101025616,model_3,05.15.2138,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,20201101030309,model_3,05.15.2138,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000,20201101030309,model_3,05.15.2138,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000,20201101050514,model_3,05.15.2138,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,20201101050515,model_3,05.15.2138,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16554659,24999,20201130172625,model_3,05.15.2138,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16554660,24999,20201130172625,model_3,05.15.2138,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
16554661,24999,20201130172631,model_3,05.15.2138,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# ID를 기준으로 groupby하여 errtype 들을 더해주어 errtype 데이터를 생성한다
err_get=train_err_incoded.groupby(['user_id'])[ 'errtype_1',
       'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6',
       'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11',
       'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16',
       'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21',
       'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26',
       'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32',
       'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37',
       'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42'].sum()

In [23]:
# 다루기 쉽게 DaraFrame으로 전환해준다
errtype_train=pd.DataFrame(err_get)
errtype_train

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
user_id,,,,,,,,,,,,,,,,,,,,,
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,...,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0


### errcode의 상위 30개 전처리 
errcode의 상위 30개가 99%이상의 수를 나타내므로 임의로 30으로 선정하였다

In [5]:
# errcode의 종류가 너무 많아 상위 30개 외의 errcode는 "other"로 변경해준다
# 그를 위해 활용할 데이터 errcode_30을 생성


errcode_30=pd.DataFrame(train_err['errcode'].value_counts()[:30]).reset_index()
errcode_30.columns=['errcode','counts']
errcode_30.head()


,errcode,counts
0,1,8906967
1,0,2599123
2,connection timeout,1835600
3,B-A8002,897863
4,80,334018


In [13]:
#errcode_30 데이터를 원본 데이터에 merge 시키면 30개에 해당하지 않는 errcode는 nan이 됨을 이용한다.

train_errcode=pd.merge(train_err,errcode_30,on='errcode',how='left')
train_errcode['counts']=train_errcode['counts'].fillna(0) # nan을 0으로 전환해준다(counts를 합쳤기에 원래는 0이 없었음)

In [14]:
#counts가 0인(30개에 들지 못하는) errcode들을 'other'로 변경해준다.¶
train_errcode['errcode'][train_errcode['counts']==0]='other'
train_errcode['errcode'].value_counts()

1                                     8906967
0                                     2599123
connection timeout                    1835600
B-A8002                                897863
80                                     334018
79                                     332422
14                                     263577
active                                 219238
2                                      166382
84                                     129876
85                                     127744
standby                                110370
NFANDROID2                             104138
connection fail to establish           100940
3                                       91223
90                                      65992
89                                      64799
other                                   38497
S-61001                                 34631
95                                      23452
94                                      22118
4                                 

In [17]:
train_errcode['errcode'].unique()

array(['1', '0', '2', 'NFANDROID2', 'B-A8002', '14', '4', '13', '3',
       '8.0', 'standby', 'active', 'Q-64002', 'connection timeout',
       'S-61001', 'other', 'connection fail to establish', '80', '79',
       '81', '86', '84', 'connectionterminated by local host', '78', '85',
       '90', '89', '95', '94', '6', '5'], dtype=object)

In [15]:
#errcode를 원핫 인코딩해주고 id를 기준으로 groupby 하여 sum을 해준다

#원핫인코딩
train_errcode_OH=pd.get_dummies(train_errcode,columns=['errcode'])
train_errcode_OH

,user_id,time,model_nm,fwver,errtype,counts,errcode_0,errcode_1,errcode_13,errcode_14,...,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_other,errcode_standby
0,10000,20201101025616,model_3,05.15.2138,15,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,20201101030309,model_3,05.15.2138,12,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000,20201101030309,model_3,05.15.2138,11,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000,20201101050514,model_3,05.15.2138,16,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,20201101050515,model_3,05.15.2138,4,2599123.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16554659,24999,20201130172625,model_3,05.15.2138,16,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16554660,24999,20201130172625,model_3,05.15.2138,4,2599123.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16554661,24999,20201130172631,model_3,05.15.2138,4,2599123.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#ID를 기준으로 groupby해주어 sum해준다
# other를 제외한 errcode들을 groupby 해줌
errcode_train=train_errcode_OH.groupby(['user_id'])['errcode_0', 'errcode_1', 'errcode_13', 'errcode_14', 'errcode_2',
       'errcode_3', 'errcode_4', 'errcode_5', 'errcode_6', 'errcode_78',
       'errcode_79', 'errcode_8.0', 'errcode_80', 'errcode_81', 'errcode_84',
       'errcode_85', 'errcode_86', 'errcode_89', 'errcode_90', 'errcode_94',
       'errcode_95', 'errcode_B-A8002', 'errcode_NFANDROID2',
       'errcode_Q-64002', 'errcode_S-61001', 'errcode_active',
       'errcode_connection fail to establish', 'errcode_connection timeout',
       'errcode_connectionterminated by local host','errcode_standby'].sum()

In [20]:
errcode_train=pd.DataFrame(errcode_train)
errcode_train

,errcode_0,errcode_1,errcode_13,errcode_14,errcode_2,errcode_3,errcode_4,errcode_5,errcode_6,errcode_78,...,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_standby
user_id,,,,,,,,,,,,,,,,,,,,,
10000,104.0,212.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,183.0,1274.0,1.0,16.0,7.0,5.0,18.0,0.0,0.0,0.0,...,0.0,53.0,56.0,0.0,0.0,126.0,0.0,0.0,0.0,625.0
10002,132.0,172.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,42.0,229.0,0.0,3.0,3.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,12.0,0.0,7.0,0.0,0.0
10004,98.0,529.0,3.0,9.0,14.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,1.0,104.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,22.0,125.0,4.0,14.0,4.0,4.0,0.0,0.0,0.0,0.0,...,0.0,2.0,7.0,0.0,0.0,4.0,0.0,0.0,0.0,8.0
24996,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,146.0,598.0,1.0,2.0,20.0,13.0,0.0,0.0,0.0,0.0,...,0.0,8.0,8.0,0.0,0.0,4.0,0.0,13.0,1.0,11.0


## errtype_train 과 errcode_train을 결합하여 err_group_ID_train 데이터 생성

In [30]:
err_group_ID_train=pd.concat([errtype_train,errcode_train],axis=1).reset_index() #결합한 후 user_id를 꺼내기 위해 reset_index()활용
err_group_ID_train


,user_id,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,...,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_standby
0,10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,...,0.0,53.0,56.0,0.0,0.0,126.0,0.0,0.0,0.0,625.0
2,10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,12.0,0.0,7.0,0.0,0.0
4,10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,1.0,104.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,...,0.0,2.0,7.0,0.0,0.0,4.0,0.0,0.0,0.0,8.0
14996,24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,...,0.0,8.0,8.0,0.0,0.0,4.0,0.0,13.0,1.0,11.0
14998,24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [31]:
#csv로 꺼내기
err_group_ID_train.to_csv(r"err_group_ID_train.csv", index=False)

## test전처리
#### train과 동일한 방식을 사용하며 errcode의 상위 30개 역시 train과 동일하게 사용한다

### errtype 전처리
방식은 train과 동일

In [6]:
# errtype 별 count를 하기위해 onehot인코딩을 해준다
test_err_incoded=pd.get_dummies(test_err,columns=['errtype'])
test_err_incoded

,user_id,time,model_nm,fwver,errcode,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,20201101030227,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000,20201101030227,model_1,04.16.3553,2,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30000,20201101030228,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30000,20201101030256,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30000,20201101030300,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16532644,44998,20201130211831,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16532645,44998,20201130211832,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16532646,44998,20201130212259,model_1,04.16.3553,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# ID를 기준으로 groupby하여 errtype 들을 더해주어 errtype 데이터를 생성한다
err_get_test=test_err_incoded.groupby(['user_id'])[ 'errtype_1',
       'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6',
       'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11',
       'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16',
       'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21',
       'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26',
       'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32',
       'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37',
       'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42'].sum()

In [9]:
# 다루기 쉽게 DaraFrame으로 전환해준다
errtype_test=pd.DataFrame(err_get_test)
errtype_test

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
user_id,,,,,,,,,,,,,,,,,,,,,
30000,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,...,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0
30001,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30002,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0
30003,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,...,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0
30004,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,...,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0
44995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0
44996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0


### errcode의 상위 30개 전처리 
train과 같은 기준을 사용하므로 앞서 생성한 errcode_30 데이터를 활용한다.

In [10]:
#errcode_30 데이터를 원본 데이터에 merge 시키면 30개에 해당하지 않는 errcode는 nan이 됨을 이용한다.

test_errcode=pd.merge(test_err,errcode_30,on='errcode',how='left')
test_errcode['counts']=test_errcode['counts'].fillna(0) # nan을 0으로 전환해준다(counts를 합쳤기에 원래는 0이 없었음)

In [11]:
#counts가 0인(30개에 들지 못하는) errcode들을 'other'로 변경해준다.¶
test_errcode['errcode'][test_errcode['counts']==0]='other'
test_errcode['errcode'].value_counts()

1                                     8750154
0                                     2565532
connection timeout                    1890632
B-A8002                                855747
80                                     326179
79                                     324186
14                                     316110
active                                 225519
2                                      155401
84                                     140642
standby                                139947
85                                     139255
connection fail to establish           109690
NFANDROID2                             108244
3                                       86451
connectionterminated by local host      84809
90                                      58820
89                                      57718
other                                   37704
S-61001                                 35026
4                                       19084
95                                

In [13]:
#errcode를 원핫 인코딩해주고 id를 기준으로 groupby 하여 sum을 해준다

#원핫인코딩
test_errcode_OH=pd.get_dummies(test_errcode,columns=['errcode'])
test_errcode_OH

,user_id,time,model_nm,fwver,errtype,counts,errcode_0,errcode_1,errcode_13,errcode_14,...,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_other,errcode_standby
0,30000,20201101030227,model_1,04.16.3553,31,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000,20201101030227,model_1,04.16.3553,33,166382.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30000,20201101030228,model_1,04.16.3553,15,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30000,20201101030256,model_1,04.16.3553,22,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30000,20201101030300,model_1,04.16.3553,11,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,2599123.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16532644,44998,20201130211831,model_1,04.16.3553,31,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16532645,44998,20201130211832,model_1,04.16.3553,15,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16532646,44998,20201130212259,model_1,04.16.3553,16,8906967.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#ID를 기준으로 groupby해주어 sum해준다
# other를 제외한 errcode들을 groupby 해줌
errcode_test=test_errcode_OH.groupby(['user_id'])['errcode_0', 'errcode_1', 'errcode_13', 'errcode_14', 'errcode_2',
       'errcode_3', 'errcode_4', 'errcode_5', 'errcode_6', 'errcode_78',
       'errcode_79', 'errcode_8.0', 'errcode_80', 'errcode_81', 'errcode_84',
       'errcode_85', 'errcode_86', 'errcode_89', 'errcode_90', 'errcode_94',
       'errcode_95', 'errcode_B-A8002', 'errcode_NFANDROID2',
       'errcode_Q-64002', 'errcode_S-61001', 'errcode_active',
       'errcode_connection fail to establish', 'errcode_connection timeout',
       'errcode_connectionterminated by local host','errcode_standby'].sum()

In [15]:
errcode_test=pd.DataFrame(errcode_test)
errcode_test

,errcode_0,errcode_1,errcode_13,errcode_14,errcode_2,errcode_3,errcode_4,errcode_5,errcode_6,errcode_78,...,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_standby
user_id,,,,,,,,,,,,,,,,,,,,,
30000,90.0,1251.0,0.0,14.0,15.0,5.0,0.0,0.0,0.0,0.0,...,0.0,53.0,0.0,0.0,9.0,141.0,55.0,1107.0,0.0,4.0
30001,98.0,180.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
30002,186.0,614.0,0.0,4.0,13.0,3.0,0.0,0.0,0.0,0.0,...,0.0,77.0,1.0,0.0,0.0,7.0,0.0,26.0,0.0,6.0
30003,91.0,147.0,0.0,11.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,106.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
30004,193.0,395.0,1.0,25.0,4.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,90.0,0.0,6.0,5.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,139.0,624.0,5.0,26.0,14.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,87.0,0.0,12.0,0.0,7.0,197.0,0.0,0.0
44995,105.0,370.0,2.0,6.0,13.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44996,251.0,698.0,3.0,4.0,529.0,573.0,6.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,104.0,4.0,44.0,0.0,11.0


## errtype_test 과 errcode_test을 결합하여 err_group_ID_test 데이터 생성

In [16]:
err_group_ID_test=pd.concat([errtype_test,errcode_test],axis=1).reset_index() #결합한 후 user_id를 꺼내기 위해 reset_index()활용
err_group_ID_test


,user_id,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,...,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_standby
0,30000,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,...,0.0,53.0,0.0,0.0,9.0,141.0,55.0,1107.0,0.0,4.0
1,30001,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30002,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,...,0.0,77.0,1.0,0.0,0.0,7.0,0.0,26.0,0.0,6.0
3,30003,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,...,0.0,106.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,30004,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,...,0.0,0.0,90.0,0.0,6.0,5.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,...,0.0,0.0,87.0,0.0,12.0,0.0,7.0,197.0,0.0,0.0
14994,44995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,...,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,44996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,104.0,4.0,44.0,0.0,11.0
14996,44997,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,...,0.0,3.0,49.0,0.0,0.0,1655.0,1010.0,10449.0,1.0,20.0


In [ ]:
# test에서 빠져있는 43262를 0벡터로 더해준다

additional_index = pd.DataFrame(np.zeros(len(err_group_ID_test.columns)).reshape(1, -1))
columns_names = err_group_ID_test.columns
additional_index.columns = columns_names
additional_index.user_id = [43262]   # 리스트로 입력 안 하면 오류 뜬다.
additional_index.index = [43262]
additional_index

err_group_ID_test = err_group_ID_test.append(additional_index)
err_group_ID_test

In [17]:
#csv로 꺼내기
err_group_ID_test.to_csv(r"err_group_ID_test.csv", index=False)

## 그룹데이터

In [40]:
train_group=pd.read_csv(r"err_group_ID_train.csv")
test_group=pd.read_csv(r"err_group_ID_test.csv").sort_values(by=['user_id'],axis=0) #마지막 추가해준 43262배치 위해서

# Target추가
ID를 기준으로target을 추가해준다

In [6]:
train_target=pd.read_csv(r"train_problem_data.csv")
train_target

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000
...,...,...
5424,20167,20201125120000
5425,16270,20201110120000
5426,19114,20201106230000
5427,21505,20201104110000


In [12]:
# traget이 시간으로 동일한 ID에도 중복해서 나타난다.
# train_group 에는 시간을 고려하지 않은 target을 더해주면된다
# 그러므로 결합해주기 전에 중복된 값을 삭제한다

train_target=pd.DataFrame(train_target.groupby(['user_id'])['time'].min().reset_index())

In [17]:
train_target.columns=['user_id','target']

In [18]:
#train_group에 train_target을  user_id 기준으로 merge 해준다

train_group_target=pd.merge(train_group,train_target,on="user_id",how='left')
train_group_target

,user_id,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,...,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_standby,target
0,10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,...,53.0,56.0,0.0,0.0,126.0,0.0,0.0,0.0,625.0,2.020111e+13
2,10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,12.0,0.0,7.0,0.0,0.0,NaN
4,10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,1.0,104.0,0.0,5.0,2.020110e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,...,2.0,7.0,0.0,0.0,4.0,0.0,0.0,0.0,8.0,NaN
14996,24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
14997,24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,...,8.0,8.0,0.0,0.0,4.0,0.0,13.0,1.0,11.0,2.020112e+13
14998,24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.020111e+13


### target값을 0과 1로 전환해준다

In [21]:
# 결측치 0으로 대체
train_group_target['target']=train_group_target['target'].fillna(0)

# 결측치(0)가 아니면 1로 대체
for x in range(len(train_group_target['target'])):  
    if train_group_target['target'][x]>0:
        train_group_target['target'][x]=1

In [30]:
train_group_target

,user_id,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,...,errcode_B-A8002,errcode_NFANDROID2,errcode_Q-64002,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_connectionterminated by local host,errcode_standby,target
0,10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,...,53.0,56.0,0.0,0.0,126.0,0.0,0.0,0.0,625.0,1.0
2,10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,12.0,0.0,7.0,0.0,0.0,0.0
4,10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,1.0,104.0,0.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,...,2.0,7.0,0.0,0.0,4.0,0.0,0.0,0.0,8.0,0.0
14996,24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,...,8.0,8.0,0.0,0.0,4.0,0.0,13.0,1.0,11.0,1.0
14998,24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


# Modeling

In [42]:
#데이터 분리
from sklearn.model_selection import StratifiedKFold, KFold,GridSearchCV, train_test_split

#모델 
from lightgbm import LGBMClassifier
import lightgbm as lgbm

#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
from sklearn.metrics import roc_auc_score


In [43]:
#run traning
LGBM=LGBMClassifier(boosting_type='gbdt',
                    objective='binary', 
                    metric='auc', 
                    random_state=0, 
                    num_boost_round = 1000)
#result=cross_val_score(LGBM,train_features, train_target,cv=10,scoring='roc_auc')
#print('The cross validated score for LightBoost is:',result.mean())

In [44]:
LGBM.fit(train_group, train_group_target["target"])

LGBMClassifier(metric='auc', num_boost_round=1000, objective='binary',
               random_state=0)

In [45]:
proba=pd.DataFrame(test_group['user_id'])
proba['problem']=LGBM.predict_proba(test_group)[:,1]
#LGBM=LGBM.drop([0,1],axis=1)
#LGBM.to_csv(r"/content/LGBM.csv", index=False)

In [46]:
proba.to_csv(r"proba_basic.csv",index=False)

In [ ]:
y=train_group_target['target']
X=train_group_target.drop('target',axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25, random_state=1)

lgbm_wrapper_time=LGBMClassifier(learning_rate=0.01,max_depth=10,)